In [2]:
import os
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install -- pathlib
!{sys.executable} -m pip install -- yfinance
!{sys.executable} -m pip install -- altair

     |████████████████████████████████| 1.5 MB 23.8 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.3
    Uninstalling pip-20.3:
      Successfully uninstalled pip-20.3
     |████████████████████████████████| 49 kB 6.3 MB/s  eta 0:00:01
  Created wheel for pathlib: filename=pathlib-1.0.1-py3-none-any.whl size=14346 sha256=4892ffa2180a345b6f3322876ed4c963d25f74ee984b4a9594692d7350b02e3d
  Stored in directory: /home/ec2-user/.cache/pip/wheels/e1/32/91/afe2cabe6f77819de11759f2a07d538cd521ef3a9dd81ba0b4
Successfully built pathlib
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=04caf264c893577509b42b322b11042a9facb5694f531bfcd0abc1685a79592d
  Stored in directory: /home/ec2-user/.cache/pip/wheels/68/72/87/a384b8b8139f2489024464d6f5937e6acb2054c57d25519fb9
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8366 sha256=8c2592dd9ca09dd95607db24d7177bdee6bd56b846e14094dc9104ead665

In [14]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
from pathlib import Path
import yfinance as yf
import altair as alt
from datetime import date
import IPython.display
import datetime

In [4]:
def dataframe_pull(net, start_date, end_date):
    #Cloudflare
    net = yf.Ticker(net)
        
    net_historical = net.history(start=start_date, end=end_date, interval="1d")
    
    temp_df = net_historical
    return (temp_df)

In [5]:
def basic_analysis_func(symbol, start_date):
    import datetime
    end_date = datetime.date.today().strftime("%Y-%m-%d")
    df = dataframe_pull(symbol, start_date, end_date).reset_index()
    df_dup = df
    
    # Additionals for each dataframe
    df['Symbol'] = symbol
    cols = df.columns
    df[cols] = df[cols].round(3)
    df['Quarters'] = df['Date'].apply(lambda x : x.quarter)
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = df['Date'].apply(lambda x : x.month)
    df['Dividend Yield (%; Quarterly)'] = round((df['Dividends'] / df['Close']), 3)
    df['Quarter Yield'] = round((df['Dividends'] / df['Close']) * 100, 2)

    
    # Visualisations
    display(Dividends_Payment_Quarterly(df, symbol) | Dividends_Yield_Quarterly(df, symbol))
    
    # Continuations of Analysis
    divs_list = df['Dividends'].unique()
    divs_list = pd.DataFrame(divs_list)
    divs_list = ((divs_list[0].loc[divs_list[0] != 0])
                 .to_numpy())
    df = ((df.loc[df['Dividends'].isin(divs_list)])
          .reset_index(drop=True))
    yearly_dividends = ((pd.DataFrame(
        df.groupby(['Year'])['Dividends'].sum()))
                        .reset_index())
    df = (pd.merge(df,
              yearly_dividends,
              on ='Year',
              how ='inner')
              .rename(columns={"Dividends_x": "Dividends", "Dividends_y": "Dividends_Annualy"}))
    if len(df['Dividends'].unique()) <= 1:
        pass
    else:
        increase_div_quarter = (((pd.DataFrame(df.groupby(['Quarters'])['Dividends'].nlargest(1))
               .reset_index())
                .sort_values('Dividends', ascending=False)
               .drop(columns=['level_1', 'Dividends']))
               .head(1)
               .reset_index(drop=True))

        quarter_div_dict = {
        4 : [12, 11, 10],
        3 : [9, 8, 7],
        2 : [6, 5, 4],
        1 : [3, 2, 1]
        }

        div_yield_df = (df[['Date', 'Close', 'Dividends_Annualy']].
                    loc[(df['Date'].apply(lambda x : x.month)
                    .isin(quarter_div_dict[int(increase_div_quarter.values)]))])

        # columns def on div_yield_df
        div_yield_df['Yield'] = (div_yield_df['Dividends_Annualy'] / div_yield_df['Close'])
        div_yield_df['Yield_%'] = div_yield_df['Yield'].map(lambda n: '{:,.2%}'.format(n))
        div_yield_df['Yield'] = (div_yield_df['Yield'] * 100)
        div_yield_df['Year'] = pd.DatetimeIndex(div_yield_df['Date']).year
        
#         print(increase_div_quarter)
        if (increase_div_quarter.values.tolist()[0][0]) % 2 == 0:
            # Issue if raise is on the same year
            # Here we will divide them into two, in order to later on compare between them
            low_df = pd.DataFrame(df.groupby(['Year'])['Dividends'].nsmallest(1).reset_index())
            low_df = low_df.drop(columns=['level_1'])

            high_df = pd.DataFrame(df.groupby(['Year'])['Dividends'].nlargest(1).reset_index())
            high_df = high_df.drop(columns=['level_1'])

            # Now we will merge the two tables
            div_growth_df = pd.merge(low_df,
                      high_df,
                      on ='Year',
                      how ='inner')
            div_growth_df = div_growth_df.rename(columns={"Dividends_x": "Dividends_prior_inc", "Dividends_y": "Dividends_post_inc"})

        else:
            # Issue if raise is on the same year
            # Here we will divide them into two, in order to later on compare between them
            low_df = pd.DataFrame(df.groupby(['Year'])['Dividends'].nsmallest(1).reset_index())
            low_df = low_df.drop(columns=['level_1'])

            high_df = pd.DataFrame(df.groupby(['Year'])['Dividends'].nlargest(1).reset_index())
            high_df = high_df.drop(columns=['level_1'])
            high_df['Year'] = high_df['Year'] + 1

            # Now we will merge the two tables
            div_growth_df = pd.merge(low_df,
                      high_df,
                      on ='Year',
                      how ='left')
            div_growth_df = div_growth_df.rename(columns={"Dividends_x": "Dividends_post_inc", "Dividends_y": "Dividends_prior_inc"})

                
#         print(div_growth_df)
        div_growth_df['Div Growth (%)'] =  round((div_growth_df['Dividends_post_inc'] / div_growth_df['Dividends_prior_inc']) - 1, 3) 
        div_growth_df['Div Growth ($)'] =  (div_growth_df['Dividends_post_inc'] - div_growth_df['Dividends_prior_inc'])
        div_growth_df['Div Growth f(%)'] = div_growth_df['Div Growth (%)'].map("{:.2%}".format)
        display(Div_payment_and_growth_viso(div_yield_df, div_growth_df, symbol))
        return(df, df_dup, div_yield_df, div_growth_df)
        
            

In [6]:
def basic_analysis_func_no_disp(symbol, start_date):
    import datetime
    end_date = datetime.date.today().strftime("%Y-%m-%d")
    df = dataframe_pull(symbol, start_date, end_date).reset_index()
    df_dup = df
    
    # Additionals for each dataframe
    df['Symbol'] = symbol
    cols = df.columns
    df[cols] = df[cols].round(3)
    df['Quarters'] = df['Date'].apply(lambda x : x.quarter)
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = df['Date'].apply(lambda x : x.month)
    df['Dividend Yield (%; Quarterly)'] = round((df['Dividends'] / df['Close']), 3)
    df['Quarter Yield'] = round((df['Dividends'] / df['Close']) * 100, 2)

    
    # Visualisations
#     display(Dividends_Payment_Quarterly(df, symbol) | Dividends_Yield_Quarterly(df, symbol))
    
    # Continuations of Analysis
    divs_list = df['Dividends'].unique()
    divs_list = pd.DataFrame(divs_list)
    divs_list = ((divs_list[0].loc[divs_list[0] != 0])
                 .to_numpy())
    df = ((df.loc[df['Dividends'].isin(divs_list)])
          .reset_index(drop=True))
    yearly_dividends = ((pd.DataFrame(
        df.groupby(['Year'])['Dividends'].sum()))
                        .reset_index())
    df = (pd.merge(df,
              yearly_dividends,
              on ='Year',
              how ='inner')
              .rename(columns={"Dividends_x": "Dividends", "Dividends_y": "Dividends_Annualy"}))
    if len(df['Dividends'].unique()) <= 1:
        pass
    else:
        increase_div_quarter = (((pd.DataFrame(df.groupby(['Quarters'])['Dividends'].nlargest(1))
               .reset_index())
                .sort_values('Dividends', ascending=False)
               .drop(columns=['level_1', 'Dividends']))
               .head(1)
               .reset_index(drop=True))

        quarter_div_dict = {
        4 : [12, 11, 10],
        3 : [9, 8, 7],
        2 : [6, 5, 4],
        1 : [3, 2, 1]
        }

        div_yield_df = (df[['Date', 'Close', 'Dividends_Annualy']].
                    loc[(df['Date'].apply(lambda x : x.month)
                    .isin(quarter_div_dict[int(increase_div_quarter.values)]))])

        # columns def on div_yield_df
        div_yield_df['Yield'] = (div_yield_df['Dividends_Annualy'] / div_yield_df['Close'])
        div_yield_df['Yield_%'] = div_yield_df['Yield'].map(lambda n: '{:,.2%}'.format(n))
        div_yield_df['Yield'] = (div_yield_df['Yield'] * 100)
        div_yield_df['Year'] = pd.DatetimeIndex(div_yield_df['Date']).year
        
#         print(increase_div_quarter)
        if (increase_div_quarter.values.tolist()[0][0]) % 2 == 0:
            # Issue if raise is on the same year
            # Here we will divide them into two, in order to later on compare between them
            low_df = pd.DataFrame(df.groupby(['Year'])['Dividends'].nsmallest(1).reset_index())
            low_df = low_df.drop(columns=['level_1'])

            high_df = pd.DataFrame(df.groupby(['Year'])['Dividends'].nlargest(1).reset_index())
            high_df = high_df.drop(columns=['level_1'])

            # Now we will merge the two tables
            div_growth_df = pd.merge(low_df,
                      high_df,
                      on ='Year',
                      how ='inner')
            div_growth_df = div_growth_df.rename(columns={"Dividends_x": "Dividends_prior_inc", "Dividends_y": "Dividends_post_inc"})

        else:
            # Issue if raise is on the same year
            # Here we will divide them into two, in order to later on compare between them
            low_df = pd.DataFrame(df.groupby(['Year'])['Dividends'].nsmallest(1).reset_index())
            low_df = low_df.drop(columns=['level_1'])

            high_df = pd.DataFrame(df.groupby(['Year'])['Dividends'].nlargest(1).reset_index())
            high_df = high_df.drop(columns=['level_1'])
            high_df['Year'] = high_df['Year'] + 1

            # Now we will merge the two tables
            div_growth_df = pd.merge(low_df,
                      high_df,
                      on ='Year',
                      how ='left')
            div_growth_df = div_growth_df.rename(columns={"Dividends_x": "Dividends_post_inc", "Dividends_y": "Dividends_prior_inc"})

                
#         print(div_growth_df)
        div_growth_df['Div Growth (%)'] =  round((div_growth_df['Dividends_post_inc'] / div_growth_df['Dividends_prior_inc']) - 1, 3) 
        div_growth_df['Div Growth ($)'] =  (div_growth_df['Dividends_post_inc'] - div_growth_df['Dividends_prior_inc'])
        div_growth_df['Div Growth f(%)'] = div_growth_df['Div Growth (%)'].map("{:.2%}".format)
#         display(Div_payment_and_growth_viso(div_yield_df, div_growth_df, symbol))
        return(df, df_dup, div_yield_df, div_growth_df)
        
            

In [7]:
def Dividends_Payment_Quarterly(df, symbol):
    base = alt.Chart(df.tail(2500), title = f"Dividends Payment Quarterly for {symbol}").encode(
        alt.X('Date:T', axis=alt.Axis(title='Date'))
    )

    line1 = base.mark_line(opacity=0.6, color='#cc0000').encode(
        alt.Y('Dividends:Q', scale=alt.Scale(domain=[0, max(df['Dividends'])*1.5]),
              axis=alt.Axis(title='Dividends Payment Quarterly', titleColor='#cc0000', format='$')),
        alt.Y2('Dividends')
        ).properties(
        height=300,
        width = 400)

    line2 = base.mark_line(opacity=0.9).encode(
        alt.Y('Close',
              axis=alt.Axis(title='Price', titleColor='#0000b3')),
        tooltip=['Symbol', 'Close']
    )

    plot = alt.layer(line1, line2).resolve_scale(
        y = 'independent'
    )
    
    return(plot)

In [8]:
def Dividends_Yield_Quarterly(df, symbol):
    base = alt.Chart(df.tail(2500), title = f"Dividends Yield Quarterly for {symbol}").encode(
        alt.X('Date:T', axis=alt.Axis(title='Date'))
    )

    line1 = base.mark_line(opacity=0.6, color='#008000').encode(
        alt.Y('Dividend Yield (%; Quarterly):Q', 
              scale=alt.Scale(domain=[0, max(df['Dividend Yield (%; Quarterly)'])*1.5]),
              axis=alt.Axis(title='Dividends Yield Quarterly', titleColor='#008000', format='%')),
        alt.Y2('Dividend Yield (%; Quarterly)')
        ).properties(
        height=300,
        width = 400)

    line2 = base.mark_line(opacity=0.9).encode(
        alt.Y('Close',
              axis=alt.Axis(title='Price', titleColor='#0000b3')),
        tooltip=['Symbol', 'Close']
    )

    plot = alt.layer(line1, line2).resolve_scale(
        y = 'independent'
    )
    
    return(plot)

In [9]:
def Div_payment_and_growth_viso(div_yield_df, div_growth_df, symbol):
    # Visualisation for the analysis
    line1 = alt.Chart(div_yield_df, title = f"Div Payment ($) for {symbol}").mark_line().encode(
        alt.X('Year:O'),
        alt.Y('Dividends_Annualy:Q', title = 'Dividends Payment ($)' ,axis=alt.Axis(format='$'), scale=alt.Scale(domain=(min(div_yield_df.Dividends_Annualy) - 1, max(div_yield_df.Dividends_Annualy) + 1)))
    ).properties(
        height=300,
        width = 250)

    line2 = alt.Chart(div_growth_df, title = f"Div Growth (%) for {symbol}").mark_line(color='#e68a00').encode(
        alt.X('Year:O'),
        alt.Y('Div Growth (%):Q', axis=alt.Axis(format='%'),
             scale=alt.Scale(domain=(0, 0.25)))
    ).properties(
        height=300,
        width = 250)

    text1 = line1.mark_text(
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text='Dividends_Annualy:Q'
    )

    text2 = line2.mark_text(
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text='Div Growth f(%):O'
    )


    x = (line1 + text1) | (line2 + text2)
    return x

In [10]:
df, df_dup, div_yield_df, div_growth_df = basic_analysis_func('KO', '2015-01-01')

alt.HConcatChart(...)

alt.HConcatChart(...)

In [15]:
# Use case of the idea for the stock of T
symbol = 'CVX'
start_date = '2015-01-01'
end_date = datetime.date.today().strftime("%Y-%m-%d")
df = dataframe_pull(symbol, start_date, end_date).reset_index()
df.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
1498,2021-01-26,0.86,0.86,0.79,0.82,636700,0,0
1499,2021-01-27,0.78,0.82,0.76,0.76,479200,0,0
1500,2021-01-28,0.82,0.82,0.76,0.78,597600,0,0
1501,2021-01-29,0.77,0.77,0.70,0.74,791800,0,0
1502,2021-02-01,0.77,0.78,0.72,0.72,282900,0,0


In [16]:
# Additionals for each dataframe
df['Symbol'] = symbol
cols = df.columns
df[cols] = df[cols].round(3)
df['Quarters'] = df['Date'].apply(lambda x : x.quarter)
df['Year'] = df['Date'].apply(lambda x : x.year)
df['Month'] = df['Date'].apply(lambda x : x.month)
df['Dividend Yield (%; Quarterly)'] = round((df['Dividends'] / df['Close']), 3)

In [17]:

# base = alt.Chart(df.tail(2500), title = f"Dividends Payment Quarterly for {symbol}").encode(
#     alt.X('Date:T', axis=alt.Axis(title='Date'))
# )

# line1 = base.mark_line(opacity=0.6, color='#cc0000').encode(
#     alt.Y('Dividends:Q', scale=alt.Scale(domain=[0, max(df['Dividends'])*1.5]),
#           axis=alt.Axis(title='Dividends Payment Quarterly', titleColor='#cc0000', format='$')),
#     alt.Y2('Dividends')
#     )

# line2 = base.mark_line(opacity=0.9).encode(
#     alt.Y('Close',
#           axis=alt.Axis(title='Price', titleColor='#0000b3')),
#     tooltip=['Symbol', 'Close']
# )

# alt.layer(line1, line2).resolve_scale(
#     y = 'independent'
# )

In [18]:

# base = alt.Chart(df.tail(2500), title = f"Dividends Yield Quarterly for {symbol}").encode(
#     alt.X('Date:T', axis=alt.Axis(title='Date'))
# )

# line1 = base.mark_line(opacity=0.6, color='#cc0000').encode(
#     alt.Y('Dividend Yield (%; Quarterly):Q', 
#           scale=alt.Scale(domain=[0, max(df['Dividend Yield (%; Quarterly)'])*1.5]),
#           axis=alt.Axis(title='Dividends Yield Quarterly', titleColor='#cc0000', format='%')),
#     alt.Y2('Dividend Yield (%; Quarterly)')
#     )

# line2 = base.mark_line(opacity=0.9).encode(
#     alt.Y('Close',
#           axis=alt.Axis(title='Price', titleColor='#0000b3')),
#     tooltip=['Symbol', 'Close']
# )

# alt.layer(line1, line2).resolve_scale(
#     y = 'independent'
# )

### Div growth & payment for stock

In [19]:
symbol = 'XOM'
df = (dataframe_pull(symbol, '2015-01-01', '2021-01-21')).reset_index()
df_dup = df
df['Symbol'] = symbol

In [20]:
divs_list = df['Dividends'].unique()
divs_list = pd.DataFrame(divs_list)
divs_list = ((divs_list[0].loc[divs_list[0] != 0])
             .to_numpy())

In [21]:
df = ((df.loc[df['Dividends'].isin(divs_list)])
      .reset_index(drop=True))

In [22]:
df['Quarter Yield'] = round((df['Dividends'] / df['Close']) * 100, 2)
df['Year'] = pd.DatetimeIndex(df['Date']).year
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol,Quarter Yield,Year
0,2015-02-06,70.834746,70.973186,69.957965,70.373283,12331200,0.69,0,XOM,0.98,2015
1,2015-05-11,68.652213,68.683232,67.194334,67.295143,11997800,0.73,0,XOM,1.08,2015
2,2015-08-11,60.277578,61.107264,59.745325,60.653282,14486400,0.73,0,XOM,1.20,2015
3,2015-11-09,65.974571,66.006154,64.584972,64.703400,13551200,0.73,0,XOM,1.13,2015
4,2016-02-09,63.721161,64.270889,62.693399,63.800831,18418100,0.73,0,XOM,1.14,2016
5,2016-05-11,71.631982,72.049753,71.278480,71.350784,11660200,0.75,0,XOM,1.05,2016
6,2016-08-10,71.124671,71.238107,69.690512,70.014618,12714800,0.75,0,XOM,1.07,2016
7,2016-11-08,69.261297,70.160475,69.065108,69.735405,9735200,0.75,0,XOM,1.08,2016
8,2017-02-08,67.634376,67.741618,66.957950,67.213676,14006300,0.75,0,XOM,1.12,2017
9,2017-05-10,68.381302,68.589477,67.989932,68.206436,9520300,0.77,0,XOM,1.13,2017


In [23]:
yearly_dividends = ((pd.DataFrame(
    df.groupby(['Year'])['Dividends'].sum()))
                    .reset_index()
                   )
yearly_dividends

,Year,Dividends
0,2015,2.88
1,2016,2.98
2,2017,3.06
3,2018,3.23
4,2019,3.43
5,2020,3.48


In [24]:
#TODO -  Check from here the XOM case

# Merging the two tables, based on the mutual year value
df = pd.merge(df,
          yearly_dividends,
          on ='Year',
          how ='inner')
df = df.rename(columns={"Dividends_x": "Dividends", "Dividends_y": "Dividends_Annualy"})
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol,Quarter Yield,Year,Dividends_Annualy
0,2015-02-06,70.834746,70.973186,69.957965,70.373283,12331200,0.69,0,XOM,0.98,2015,2.88
1,2015-05-11,68.652213,68.683232,67.194334,67.295143,11997800,0.73,0,XOM,1.08,2015,2.88
2,2015-08-11,60.277578,61.107264,59.745325,60.653282,14486400,0.73,0,XOM,1.20,2015,2.88
3,2015-11-09,65.974571,66.006154,64.584972,64.703400,13551200,0.73,0,XOM,1.13,2015,2.88
4,2016-02-09,63.721161,64.270889,62.693399,63.800831,18418100,0.73,0,XOM,1.14,2016,2.98


In [25]:
# Todo - fix if year is prior
# Deciding which month is the month with the biggest dividend
df['Quarter'] = pd.DatetimeIndex(df['Date']).quarter
increase_div_quarter = (((pd.DataFrame(df.groupby(['Quarter'])['Dividends'].nlargest(1))
           .reset_index())
            .sort_values('Dividends', ascending=False)
           .drop(columns=['level_1', 'Dividends']))
           .head(1)
           .reset_index(drop=True))
increase_div_quarter

,Quarter
0,1


In [26]:
quarter_div_dict = {
    4 : [12, 11, 10],
    3 : [9, 8, 7],
    2 : [6, 5, 4],
    1 : [3, 2, 1]
}

div_yield_df = (df[['Date', 'Close', 'Dividends_Annualy']].
                    loc[(df['Date'].apply(lambda x : x.month)
                    .isin(quarter_div_dict[int(increase_div_quarter.values)]))])    
div_yield_df.tail()

,Date,Close,Dividends_Annualy
4,2016-02-09,63.800831,2.98
8,2017-02-08,67.213676,3.06
12,2018-02-09,64.960831,3.23
16,2019-02-08,66.103302,3.43
20,2020-02-10,56.322723,3.48


In [27]:
div_yield_df['Yield'] = (div_yield_df['Dividends_Annualy'] / div_yield_df['Close'])
div_yield_df.tail()

,Date,Close,Dividends_Annualy,Yield
4,2016-02-09,63.800831,2.98,0.046708
8,2017-02-08,67.213676,3.06,0.045526
12,2018-02-09,64.960831,3.23,0.049722
16,2019-02-08,66.103302,3.43,0.051888
20,2020-02-10,56.322723,3.48,0.061787


In [28]:
div_yield_df['Yield_%'] = div_yield_df['Yield'].map(lambda n: '{:,.2%}'.format(n))
div_yield_df['Yield'] = (div_yield_df['Yield'] * 100)
div_yield_df['Year'] = pd.DatetimeIndex(div_yield_df['Date']).year
div_yield_df.tail()

,Date,Close,Dividends_Annualy,Yield,Yield_%,Year
4,2016-02-09,63.800831,2.98,4.670786,4.67%,2016
8,2017-02-08,67.213676,3.06,4.552645,4.55%,2017
12,2018-02-09,64.960831,3.23,4.972227,4.97%,2018
16,2019-02-08,66.103302,3.43,5.188848,5.19%,2019
20,2020-02-10,56.322723,3.48,6.178678,6.18%,2020


In [29]:
        if (increase_div_quarter.values.tolist()[0][0]) % 2 == 0:
            # Issue if raise is on the same year
            # Here we will divide them into two, in order to later on compare between them
            low_df = pd.DataFrame(df.groupby(['Year'])['Dividends'].nsmallest(1).reset_index())
            low_df = low_df.drop(columns=['level_1'])

            high_df = pd.DataFrame(df.groupby(['Year'])['Dividends'].nlargest(1).reset_index())
            high_df = high_df.drop(columns=['level_1'])

            # Now we will merge the two tables
            div_growth_df = pd.merge(low_df,
                      high_df,
                      on ='Year',
                      how ='inner')
            div_growth_df = div_growth_df.rename(columns={"Dividends_x": "Dividends_prior_inc", "Dividends_y": "Dividends_post_inc"})

        else:
            # Issue if raise is on the same year
            # Here we will divide them into two, in order to later on compare between them
            low_df = pd.DataFrame(df.groupby(['Year'])['Dividends'].nsmallest(1).reset_index())
            low_df = low_df.drop(columns=['level_1'])

            high_df = pd.DataFrame(df.groupby(['Year'])['Dividends'].nlargest(1).reset_index())
            high_df = high_df.drop(columns=['level_1'])
            high_df['Year'] = high_df['Year'] + 1

            # Now we will merge the two tables
            div_growth_df = pd.merge(low_df,
                      high_df,
                      on ='Year',
                      how ='left')
            div_growth_df = div_growth_df.rename(columns={"Dividends_x": "Dividends_post_inc", "Dividends_y": "Dividends_prior_inc"})

In [30]:
div_growth_df['Div Growth (%)'] =  round((div_growth_df['Dividends_post_inc'] / div_growth_df['Dividends_prior_inc']) - 1, 3) 
div_growth_df.tail()

,Year,Dividends_post_inc,Dividends_prior_inc,Div Growth (%)
1,2016,0.73,0.73,0.0
2,2017,0.75,0.75,0.0
3,2018,0.77,0.77,0.0
4,2019,0.82,0.82,0.0
5,2020,0.87,0.87,0.0


In [31]:
div_growth_df['Div Growth ($)'] =  (div_growth_df['Dividends_post_inc'] - div_growth_df['Dividends_prior_inc'])
div_growth_df.tail()

,Year,Dividends_post_inc,Dividends_prior_inc,Div Growth (%),Div Growth ($)
1,2016,0.73,0.73,0.0,0.0
2,2017,0.75,0.75,0.0,0.0
3,2018,0.77,0.77,0.0,0.0
4,2019,0.82,0.82,0.0,0.0
5,2020,0.87,0.87,0.0,0.0


In [32]:
div_growth_df['Div Growth f(%)'] = div_growth_df['Div Growth (%)'].map("{:.2%}".format)
div_growth_df

,Year,Dividends_post_inc,Dividends_prior_inc,Div Growth (%),Div Growth ($),Div Growth f(%)
0,2015,0.69,NaN,NaN,NaN,nan%
1,2016,0.73,0.73,0.0,0.0,0.00%
2,2017,0.75,0.75,0.0,0.0,0.00%
3,2018,0.77,0.77,0.0,0.0,0.00%
4,2019,0.82,0.82,0.0,0.0,0.00%
5,2020,0.87,0.87,0.0,0.0,0.00%


In [33]:
# Visualisation for the analysis
line1 = alt.Chart(div_yield_df, title = f"Div Payment ($) for {symbol}").mark_line().encode(
    alt.X('Year:O'),
    alt.Y('Dividends_Annualy:Q', title = 'Dividends Payment ($)' ,axis=alt.Axis(format='$'), scale=alt.Scale(domain=(min(div_yield_df.Dividends_Annualy) - 1, max(div_yield_df.Dividends_Annualy) + 1)))
).properties(
    height=300,
    width = 150)

line2 = alt.Chart(div_growth_df, title = f"Div Growth (%) for {symbol}").mark_line(color='#e68a00').encode(
    alt.X('Year:O'),
    alt.Y('Div Growth (%):Q', axis=alt.Axis(format='%')),
).properties(
    height=300,
    width = 150)

text1 = line1.mark_text(
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Dividends_Annualy:Q'
)

text2 = line2.mark_text(
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Div Growth f(%):O'
)


(line1 + text1) | (line2 + text2)

alt.HConcatChart(...)

### Comparison of few diffrent investments - to be completed

Need to create a function that calculates the amount of money entered,
and the amount you will earn on it in the next 10 years, according to the current dividend yield.

In [34]:
def financial_data(symbol):
    temp_df = pd.read_html(f'https://finance.yahoo.com/quote/{symbol}?p={symbol}&.tsrc=fin-srch')
    temp_df = (temp_df[0]).append(temp_df[1])
    return temp_df

# Function that calls for related data of each company named within the brackets

In [69]:
def further_analysis(symbol):
    financial_df = financial_data(symbol)
    
    # Playing with the data frame
    financial_df = (pd.DataFrame((np.array([financial_df[1].values.tolist()])), 
                             columns=(financial_df[0].values.tolist())))
    financial_df = financial_df.drop(['Open', 'Bid', 'Ask', "Day's Range", "Volume", "Avg. Volume"], axis=1)
#     financial_df['52 Week Range'].values.tolist()

    values_lst = financial_df['52 Week Range'].apply(lambda x : x.split(" - "))
    n1 = (int ((values_lst[0][0]).replace(".", "").
           replace(",", ""))
          / 100
            )
    n2 = (int ((values_lst[0][1]).replace(".", "").
           replace(",", ""))
          / 100
            )
    if n1 > n2:
        financial_df['52 Weeks High'] = n1
        financial_df['52 Weeks Low'] = n2
    else:
        financial_df['52 Weeks High'] = n2
        financial_df['52 Weeks Low'] = n1
        
    financial_df = financial_df.drop(["52 Week Range"], axis=1)
    financial_df['Price precentile (between low & high)'] = round(string_to_int_yahoo(financial_df['Previous Close'][0]) / 
                                     ((financial_df['52 Weeks High'] + financial_df['52 Weeks Low']) / 2), 3)
    
    assets = ""
    if ('Market Cap' in financial_df.columns):
        assets = 'Market Cap'
        financial_df['Market Cap'] = financial_df['Market Cap'].apply(lambda x : float(x[:-1]))
        financial_df['Market Cap'] = round(financial_df['Market Cap'], 1)
    else:
        assets = 'Net Assets'
        financial_df['Net Assets'] = financial_df[f'{assets}'].apply(lambda x : float(x[:-1]))
        financial_df['Net Assets'] = round(financial_df[f'{assets}'], 1)
        
    div_growth_df['Div Growth (%)'].mean()
    financial_df['5 YR Avg Div. Growth'] = round(div_growth_df['Div Growth (%)'].mean(), 3)
    financial_df['5 YR Avg Div. Growth (f%)'] = financial_df['5 YR Avg Div. Growth'].map("{:.2%}".format)
    df.head()
    latest_div = df.tail(1)
    financial_df['Current Yield'] = (round((latest_div['Dividends'] * 4) / latest_div['Close'], 3)).values
    financial_df['Chowder Index (5 YR Yield + Current Yield)'] = financial_df['Current Yield'] + financial_df['5 YR Avg Div. Growth']
    financial_df['Chowder Index f(5 YR Yield + Current Yield)'] = financial_df['Chowder Index (5 YR Yield + Current Yield)'].map("{:.2%}".format)
    
    # for Index funds
    if (financial_df.columns.tolist()[1] == 'Net Assets'):
        financial_table = financial_df[['Net Assets', 'Beta (5Y Monthly)', 'PE Ratio (TTM)', '52 Weeks High', '52 Weeks Low', 'Price precentile (between low & high)', '5 YR Avg Div. Growth (f%)', '5 YR Avg Div. Growth', 'Chowder Index f(5 YR Yield + Current Yield)', 'Chowder Index (5 YR Yield + Current Yield)']]
    else:
        financial_table = financial_df[[f'{assets}', 'Beta (5Y Monthly)', 'PE Ratio (TTM)', '52 Weeks High', '52 Weeks Low', 'Price precentile (between low & high)', '5 YR Avg Div. Growth (f%)', '5 YR Avg Div. Growth', 'Chowder Index f(5 YR Yield + Current Yield)', 'Chowder Index (5 YR Yield + Current Yield)']]
        
    five_yr_div = financial_table[['5 YR Avg Div. Growth', '5 YR Avg Div. Growth (f%)']]
    five_yr_div = pd.DataFrame(five_yr_div).rename(columns={'5 YR Avg Div. Growth' : '1', '5 YR Avg Div. Growth (f%)' : '2'})

    chowder_index = financial_table[['Chowder Index (5 YR Yield + Current Yield)', 'Chowder Index f(5 YR Yield + Current Yield)']]
    chowder_index = pd.DataFrame(chowder_index).rename(columns={'Chowder Index (5 YR Yield + Current Yield)' : '1', 'Chowder Index f(5 YR Yield + Current Yield)' : '2'})

    display(fundamental_analysis(financial_table, five_yr_div, chowder_index, assets))
    

In [70]:
def string_to_int_yahoo(value):
    value = (int((value.replace(".", "").
       replace(",", ""))
        ) / 100 )
    return value

In [71]:
def fundamental_analysis(financial_table, five_yr_div, chowder_index, assets):
    beta = alt.Chart(financial_table, title="Beta").mark_bar(size=25).encode(
        alt.Y('Beta (5Y Monthly):Q',
            scale=alt.Scale(domain=(0, 2)), title=None)).properties(width=40)

    source = pd.DataFrame({
        'goal': [1]
    })

    tick1 = alt.Chart(source).mark_tick(
        color='red',
        thickness=2,
        size=40 * 0.9,  # controls width of tick.
    ).encode(
        y='goal'
    )

    text1 = alt.Chart(financial_table).mark_text(
        align='center',
        baseline='middle',
        dy=-5
    ).encode(
        y=alt.Y('Beta (5Y Monthly):Q'),
        text='Beta (5Y Monthly)'
    )

    market_cap = alt.Chart(financial_table, title=f'{assets} (in Billions)').mark_bar(size=25).encode(
        alt.Y(f'{assets}:Q',
            scale=alt.Scale(domain=(0, 500)), title=None))
    
    text2 = alt.Chart(financial_table).mark_text(
        align='center',
        baseline='middle',
        dy=-5
    ).encode(
        y=alt.Y(f'{assets}:Q'),
        text=f'{assets}'
    )

    pe_ratio = alt.Chart(financial_table, title='PE ratio').mark_bar(size=25).encode(
        alt.Y('PE Ratio (TTM):Q',
            scale=alt.Scale(domain=(0, 100)), title=None))
    
    text3 = alt.Chart(financial_table).mark_text(
        align='center',
        baseline='middle',
        dy=-5
    ).encode(
        y=alt.Y('PE Ratio (TTM):Q'),
        text='PE Ratio (TTM)'
    )
    
    financial_table['Price precentile (between low & high) %'] = (financial_table['Price precentile (between low & high)']
                                           .map(lambda n: '{:,.2%}'.format(n)))
    price_perc = alt.Chart(financial_table, title='Price Precentile (Yearly)').mark_bar(size=25).encode(
        alt.Y('Price precentile (between low & high):Q',
            scale=alt.Scale(domain=(0, 2)), axis=alt.Axis(format='%'), title=None))

    source = pd.DataFrame({
        'goal': [1]
    })

    tick4 = alt.Chart(source).mark_tick(
        color='#00e600',
        thickness=2,
        size=40 * 0.9,  # controls width of tick.
    ).encode(
        y='goal'
    )
    
    text4 = alt.Chart(financial_table).mark_text(
        align='center',
        baseline='middle',
        dy=-5
    ).encode(
        y=alt.Y('Price precentile (between low & high):Q'),
        text='Price precentile (between low & high) %'
    )

    #Five year div growth
    five_yr_div['1 %'] = (five_yr_div['1'].map(lambda n: '{:,.2%}'.format(n)))
    five_yr_div_growth = alt.Chart(five_yr_div, title="5 Year Avg Div. Growth (%)").mark_bar(size=25).encode(
        alt.Y('1:Q',
            scale=alt.Scale(domain=(0, 0.5)), axis=alt.Axis(format='%'), title=None))
    
    text5 = alt.Chart(five_yr_div).mark_text(
        align='center',
        baseline='middle',
        dy=-5
    ).encode(
        y=alt.Y('1:Q'),
        text='1 %'
    )

    #Chowder
    chowder_index['1 %'] = (chowder_index['1'].map(lambda n: '{:,.2%}'.format(n)))
    chowder = alt.Chart(chowder_index, title="Chowder Index").mark_bar(size=25).encode(
        alt.Y('1:Q',
            scale=alt.Scale(domain=(0, 0.5)), axis=alt.Axis(format='%'), title=None))

    source = pd.DataFrame({
        'goal': [0.1]
    })

    tick6 = alt.Chart(source).mark_tick(
        color='#00e600',
        thickness=2,
        size=40 * 0.9,  # controls width of tick.
    ).encode(
        y='goal'
    )
    
    text6 = alt.Chart(chowder_index).mark_text(
        align='center',
        baseline='middle',
        dy=-5
    ).encode(
        y=alt.Y('1:Q'),
        text='1 %'
    )

    plot = ((beta + tick1 + text1) | (market_cap + text2) | (pe_ratio + text3) | (price_perc + tick4 + text4) | (five_yr_div_growth + text5) | (chowder + tick6 + text6))
    return(plot)

In [72]:
further_analysis('SCHD')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


alt.HConcatChart(...)

#### Getting the data for each stock

In [39]:
# financial_df = (pd.DataFrame((np.array([financial_df[1].values.tolist()])), 
#                              columns=(financial_df[0].values.tolist())))
# financial_df = financial_df.drop(['Open', 'Bid', 'Ask', "Day's Range", "Volume", "Avg. Volume"], axis=1)
# financial_df

In [40]:
# import re
# financial_df['52 Week Range'].values.tolist()
# # # list(map(lambda x: '-'.join(re.findall(r'(\S\w+)\s+\w+', x)) ,financial_df['52 Week Range'].values.tolist()))

In [41]:
# values_lst = financial_df['52 Week Range'].apply(lambda x : x.split(" - "))

# # Defining which one of the two is the lower and the higher
# n1 = (int ((values_lst[0][0]).replace(".", "").
#        replace(",", ""))
#       / 100
#         )
# n2 = (int ((values_lst[0][1]).replace(".", "").
#        replace(",", ""))
#       / 100
#         )

# if n1 > n2:
#     financial_df['52 Weeks High'] = n1
#     financial_df['52 Weeks Low'] = n2
# else:
#     financial_df['52 Weeks High'] = n2
#     financial_df['52 Weeks Low'] = n1
    
# financial_df = financial_df.drop(["52 Week Range"], axis=1)
    
# # With 52 weeks high and low
# financial_df


In [42]:
# def string_to_int_yahoo(value):
#     value = (int((value.replace(".", "").
#        replace(",", ""))
#         ) / 100 )
#     return value

In [43]:
# # Price avg for this years prices (todays_price / average(52_weeks_low + 52_weeks_high))

# financial_df['Price precentile (between low & high)'] = round(string_to_int_yahoo(financial_df['Previous Close'][0]) / 
#                                      ((financial_df['52 Weeks High'] + financial_df['52 Weeks Low']) / 2), 3)
# financial_df

In [44]:
# # Market Cap re assigning
# financial_df['Market Cap'] = financial_df['Market Cap'].apply(lambda x : float(x[:-1]))
# financial_df['Market Cap']

In [45]:
# # Chowder index = Current Yield + 5 Year Growth
# div_growth_df['Div Growth (%)'].mean()

In [46]:
# financial_df['5 YR Avg Div. Growth'] = round(div_growth_df['Div Growth (%)'].mean(), 3)
# financial_df['5 YR Avg Div. Growth (f%)'] = financial_df['5 YR Avg Div. Growth'].map("{:.2%}".format)
# financial_df

In [47]:
# latest_div = df.tail(1)
# financial_df['Current Yield'] = (round((latest_div['Dividends'] * 4) / latest_div['Close'], 3)).values
# financial_df

In [48]:
# financial_df['Chowder Index (5 YR Yield + Current Yield)'] = financial_df['Current Yield'] + financial_df['5 YR Avg Div. Growth']
# financial_df['Chowder Index f(5 YR Yield + Current Yield)'] = financial_df['Chowder Index (5 YR Yield + Current Yield)'].map("{:.2%}".format)
# financial_df

In [49]:
# # for Index funds
# if (financial_df.columns.tolist()[1] == 'Net Assets'):
#     financial_table = financial_df[['Net Assets', 'Beta (5Y Monthly)', 'PE Ratio (TTM)', '52 Weeks High', '52 Weeks Low', 'Price precentile (between low & high)', '5 YR Avg Div. Growth (f%)', '5 YR Avg Div. Growth', 'Chowder Index f(5 YR Yield + Current Yield)', 'Chowder Index (5 YR Yield + Current Yield)']]
# else:
#     financial_table = financial_df[['Market Cap', 'Beta (5Y Monthly)', 'PE Ratio (TTM)', '52 Weeks High', '52 Weeks Low', 'Price precentile (between low & high)', '5 YR Avg Div. Growth (f%)', '5 YR Avg Div. Growth', 'Chowder Index f(5 YR Yield + Current Yield)', 'Chowder Index (5 YR Yield + Current Yield)']]
# financial_table

In [50]:
# five_yr_div = financial_table[['5 YR Avg Div. Growth', '5 YR Avg Div. Growth (f%)']]
# five_yr_div = pd.DataFrame(five_yr_div).rename(columns={'5 YR Avg Div. Growth' : '1', '5 YR Avg Div. Growth (f%)' : '2'})

# chowder_index = financial_table[['Chowder Index (5 YR Yield + Current Yield)', 'Chowder Index f(5 YR Yield + Current Yield)']]
# chowder_index = pd.DataFrame(chowder_index).rename(columns={'Chowder Index (5 YR Yield + Current Yield)' : '1', 'Chowder Index f(5 YR Yield + Current Yield)' : '2'})


In [51]:
# beta = alt.Chart(financial_table, title="Beta").mark_bar(size=25).encode(
#     alt.Y('Beta (5Y Monthly):Q',
#         scale=alt.Scale(domain=(0, 2)), title=None)).properties(width=40)

# source = pd.DataFrame({
#     'goal': [1]
# })

# tick1 = alt.Chart(source).mark_tick(
#     color='red',
#     thickness=2,
#     size=40 * 0.9,  # controls width of tick.
# ).encode(
#     y='goal'
# )

# text1 = alt.Chart(financial_table).mark_text(
#     align='center',
#     baseline='middle',
#     dy=-5
# ).encode(
#     y=alt.Y('Beta (5Y Monthly):Q'),
#     text='Beta (5Y Monthly)'
# )

# market_cap = alt.Chart(financial_table, title='Market Cap (in Billions)').mark_bar(size=25).encode(
#     alt.Y('Market Cap:Q',
#         scale=alt.Scale(domain=(0, 500)), title=None))

# text2 = alt.Chart(financial_table).mark_text(
#     align='center',
#     baseline='middle',
#     dy=-5
# ).encode(
#     y=alt.Y('Market Cap:Q'),
#     text='Market Cap'
# )

# pe_ratio = alt.Chart(financial_table, title='PE ratio').mark_bar(size=25).encode(
#     alt.Y('PE Ratio (TTM):Q',
#         scale=alt.Scale(domain=(0, 100)), title=None))

# text3 = alt.Chart(financial_table).mark_text(
#     align='center',
#     baseline='middle',
#     dy=-5
# ).encode(
#     y=alt.Y('PE Ratio (TTM):Q'),
#     text='PE Ratio (TTM)'
# )

# price_perc = alt.Chart(financial_table, title='Price Precentile (Yearly)').mark_bar(size=25).encode(
#     alt.Y('Price precentile (between low & high):Q',
#         scale=alt.Scale(domain=(0, 2)), title=None))

# source = pd.DataFrame({
#     'goal': [1]
# })

# tick4 = alt.Chart(source).mark_tick(
#     color='#00e600',
#     thickness=2,
#     size=40 * 0.9,  # controls width of tick.
# ).encode(
#     y='goal'
# )

# text4 = alt.Chart(financial_table).mark_text(
#     align='center',
#     baseline='middle',
#     dy=-5
# ).encode(
#     y=alt.Y('Price precentile (between low & high):Q'),
#     text='Price precentile (between low & high)'
# )

# #Five year div growth
# five_yr_div_growth = alt.Chart(five_yr_div, title="5 Year Avg Div. Growth (%)").mark_bar(size=25).encode(
#     alt.Y('1:Q',
#         scale=alt.Scale(domain=(0, 0.5)), title=None))

# text5 = alt.Chart(five_yr_div).mark_text(
#     align='center',
#     baseline='middle',
#     dy=-5
# ).encode(
#     y=alt.Y('1:Q'),
#     text='1'
# )

# #Chowder
# chowder = alt.Chart(chowder_index, title="Chowder Index").mark_bar(size=25).encode(
#     alt.Y('1:Q',
#         scale=alt.Scale(domain=(0, 0.5)), title=None))

# source = pd.DataFrame({
#     'goal': [0.1]
# })

# tick6 = alt.Chart(source).mark_tick(
#     color='#00e600',
#     thickness=2,
#     size=40 * 0.9,  # controls width of tick.
# ).encode(
#     y='goal'
# )

# text6 = alt.Chart(chowder_index).mark_text(
#     align='center',
#     baseline='middle',
#     dy=-5
# ).encode(
#     y=alt.Y('1:Q'),
#     text='1'
# )

# plot = ((beta + tick1 + text1) | (market_cap + text2) | (pe_ratio + text3) | (price_perc + tick4 + text4) | (five_yr_div_growth + text5) | (chowder + tick6 + text6))
# plot

In [52]:
# Investment_df = (div_yield_df[['Year', 'Dividends_Annualy', 'Close']]).reset_index(drop=True)
# Investment_df[['Div Growth (%)']] = div_growth_df[['Div Growth (%)']]
# Investment_df

In [126]:
import datetime

def calc_yield_func(monvey_invested, date_of_purchase, symbol):
    end_date = datetime.date.today().strftime("%Y-%m-%d")
    df_dup = dataframe_pull(symbol, date_of_purchase, end_date).reset_index()
    date_of_purchase = df_dup['Date'][0].date().strftime("%Y-%m-%d")
    if len(df_dup['Dividends'].unique()) > 1:
#         print(df_dup.head())
        amt_stocks_own = (round(monvey_invested / 
                          ((df_dup['Close'].loc[df_dup['Date'] == date_of_purchase])
                           .values
                           .tolist())[0]))

        price_stock_purchase = round(df_dup['Close'].loc[df_dup['Date'] == date_of_purchase]
                                .values
                                .tolist()[0], 2)
        stock_price_today = (df_dup['Close'].tail(1)
                             .values
                             .tolist()[0])
        stock_price_yield = round((stock_price_today / price_stock_purchase) * 100 - 100, 3)
        stock_price_diff = round(stock_price_today - price_stock_purchase, 2)
        tot_stock_price_diff = stock_price_diff * amt_stocks_own

        dividend_payed_per_stock = (df_dup['Dividends'].loc[df_dup['Date'] > date_of_purchase]).sum()
        tot_dividend_payed = round(amt_stocks_own * dividend_payed_per_stock, 2)

        dividends_yield = round(tot_dividend_payed / monvey_invested * 100, 2)
        tot_yield = ((round(tot_dividend_payed + tot_stock_price_diff, 2) + monvey_invested) / monvey_invested * 100) - 100

        gen_details = (f"You own {amt_stocks_own} stocks, which you bought for {price_stock_purchase}$.")
        ans1 = (f"Total money recieved in Dividened since invested: {tot_dividend_payed}$.")
        ans2 = (f"Total yield recieved on inital invetments, from dividend payment: {dividends_yield}%.")
        ans3 = (f"Total profit on stock's price: {round(tot_stock_price_diff, 2)}$")
        ans4 = (f"Total yield on stock's price: {stock_price_yield}%")
        ans5 = (f"Total profit from investemnt: {round(tot_dividend_payed + tot_stock_price_diff, 2)}$.")
        ans6 = (f"Total yield in % from investemnt: {round(tot_yield, 2)}%.")
        ans_lst = [gen_details, ans1, ans2, ans3, ans4, ans5, ans6]
        for i in ans_lst:
            print(i)

    else:
        amt_stocks_own = (round(monvey_invested / 
                  ((df_dup['Close'].loc[df_dup['Date'] == date_of_purchase])
                   .values
                   .tolist())[0]))

        price_stock_purchase = round(df_dup['Close'].loc[df_dup['Date'] == date_of_purchase]
                                .values
                                .tolist()[0], 2)
        stock_price_today = (df_dup['Close'].tail(1)
                             .values
                             .tolist()[0])
        stock_price_yield = round((stock_price_today / price_stock_purchase) * 100 - 100, 3)
        stock_price_diff = round(stock_price_today - price_stock_purchase, 2)
        tot_stock_price_diff = stock_price_diff * amt_stocks_own

#         dividend_payed_per_stock = (div_pay_df['Dividends'].loc[div_pay_df['Date'] > date_of_purchase]).sum()
#         tot_dividend_payed = round(amt_stocks_own * dividend_payed_per_stock, 2)

#         dividends_yield = round(tot_dividend_payed / monvey_invested * 100, 2)
        tot_yield = ((round(tot_stock_price_diff, 2) + monvey_invested) / monvey_invested * 100) - 100

        gen_details = (f"You own {amt_stocks_own} stocks, which you bought for {price_stock_purchase}$.")
#         ans1 = (f"Total money recieved in Dividened since invested: {tot_dividend_payed}$.")
#         ans2 = (f"Total yield recieved on inital invetments, from dividend payment: {dividends_yield}%.")
        ans3 = (f"Total profit on stock's price: {round(tot_stock_price_diff, 2)}$")
        ans4 = (f"Total yield on stock's price: {stock_price_yield}%")
        ans5 = (f"Total profit from investemnt: {round(tot_stock_price_diff, 2)}$.")
        ans6 = (f"Total yield in % from investemnt: {round(tot_yield, 2)}%.")
        ans_lst = [gen_details, ans3, ans4, ans5, ans6]
        for i in ans_lst:
            print(i)
    
#     for i in ans_lst:
#         print(i)
    return gen_details, ans3, ans4, ans5, ans6


In [127]:
calc_yield_func(2000, '2015-09-10', 'T')

You own 84 stocks, which you bought for 23.91$.
Total money recieved in Dividened since invested: 923.16$.
Total yield recieved on inital invetments, from dividend payment: 46.16%.
Total profit on stock's price: 386.4$
Total yield on stock's price: 19.239%
Total profit from investemnt: 1309.56$.
Total yield in % from investemnt: 65.48%.


('You own 84 stocks, which you bought for 23.91$.',
 "Total profit on stock's price: 386.4$",
 "Total yield on stock's price: 19.239%",
 'Total profit from investemnt: 1309.56$.',
 'Total yield in % from investemnt: 65.48%.')

In [100]:
# Assessed next dividend pay-date.

def next_pay_date(symbol, start_date):
    end_date = datetime.date.today().strftime("%Y-%m-%d")
    df = dataframe_pull(symbol, start_date, end_date).reset_index()
    
    next_div_df = df
    divs_list = next_div_df['Dividends'].unique()
    divs_list = pd.DataFrame(divs_list)
    divs_list = ((divs_list[0].loc[divs_list[0] != 0])
                 .to_numpy())
    next_div_df = ((next_div_df.loc[next_div_df['Dividends'].isin(divs_list)]).reset_index(drop=True))
    next_div_df['Date'] = pd.to_datetime(next_div_df['Date'])
    next_div_df['Month'] = next_div_df['Date'].apply(lambda x : x.month)
    next_div_df['Day'] = next_div_df['Date'].apply(lambda x : x.day)
    next_div_df = pd.DataFrame(next_div_df.groupby(["Month"])['Day'].mean().astype(int))
    next_div_df = next_div_df.reset_index()

    day = datetime.datetime.today().day
    month = datetime.datetime.today().month
    year = datetime.datetime.today().year

    if len(divs_list) > 1:
        for i in range(4):
            if (next_div_df['Month'][i] > month):
                day = int(next_div_df['Day'][i])
                message = f"Next dividend approximated pay-day is : {next_div_df['Day'][i]}-{next_div_df['Month'][i]}-{year}"
                print(message)
                break
            elif (next_div_df['Month'][i] == month) and (next_div_df['Day'][i] > day):
                day = int(next_div_df['Day'][i])
                message = f"Next dividend approximated pay-day is : {next_div_df['Day'][i]}-{next_div_df['Month'][i]}-{year}"
                print(message)
                break
            else:
                pass
    else:
        pass
            
next_pay_date('XOM', '2015-01-02')


Next dividend approximated pay-day is : 8-2-2021


In [101]:
# next_div_df = dataframe_pull('XOM', '2015-01-01', '2021-02-02').reset_index()
# divs_list = next_div_df['Dividends'].unique()
# divs_list = pd.DataFrame(divs_list)
# divs_list = ((divs_list[0].loc[divs_list[0] != 0])
#              .to_numpy())
# next_div_df = ((next_div_df.loc[next_div_df['Dividends'].isin(divs_list)]).reset_index(drop=True))
# next_div_df['Date'] = pd.to_datetime(next_div_df['Date'])
# next_div_df['Month'] = next_div_df['Date'].apply(lambda x : x.month)
# next_div_df['Day'] = next_div_df['Date'].apply(lambda x : x.day)
# next_div_df = pd.DataFrame(next_div_df.groupby(["Month"])['Day'].mean().astype(int))
# next_div_df = next_div_df.reset_index()

# day = datetime.datetime.today().day
# month = datetime.datetime.today().month
# year = datetime.datetime.today().year

# if len(divs_list) > 1:
#     for i in range(4):
#         if (next_div_df['Month'][i] > month):
#             day = int(next_div_df['Day'][i])
#             message = f"Next dividend approximated pay-day is : {next_div_df['Day'][i]}-{next_div_df['Month'][i]}-{year}"
#             print(message)
#             break
#         elif (next_div_df['Month'][i] == month) and (next_div_df['Day'][i] > day):
#             print(2)
#             day = int(next_div_df['Day'][i])
#             message = f"Next dividend approximated pay-day is : {next_div_df['Day'][i]}-{next_div_df['Month'][i]}-{year}"
#             print(message)
#             break
#         else:
#             pass
        

In [102]:
def correlation_analysis_365(symbol_a, symbol_b):
    start_date = (datetime.date.today() - datetime.timedelta(days=365)).strftime("%Y-%m-%d")
    end_date = datetime.date.today().strftime("%Y-%m-%d")
    df = dataframe_pull(symbol_a, start_date, end_date).reset_index()
    
    df['Symbol'] = symbol_a
    length = len(df)

    end_date = datetime.date.today().strftime("%Y-%m-%d")
    df_symbol_b = dataframe_pull(symbol_b, start_date, end_date).reset_index()
    df_symbol_b['Symbol'] = symbol_b
    df_symbol_b = df_symbol_b.tail(length)

    merged_df = ((pd.merge(df,
          df_symbol_b,
          on ='Date',
          how ='inner')))

    # Setting Corr related df
    temp_merged_df = (merged_df.drop(columns=['Open_x', 'High_x', 'Low_x', 'Volume_x', 'Dividends_x'
                         , 'Open_y', 'High_y', 'Symbol_x', 'Low_y', 'Volume_y', 'Dividends_y',
                         'Stock Splits_y', 'Stock Splits_x', 'Symbol_y']))

    temp_merged_df['Week'] = temp_merged_df['Date'].apply(lambda x : x.week)
    gen_corr = temp_merged_df.corr()['Close_x']['Close_y']

    d = {'Date': [], 'Week': [] ,'Corr': []}
    corr_df = pd.DataFrame(data=d)

    x = 0
    for i in range(0, length):
        x += 4
        corr_value = round((temp_merged_df.iloc[x:x+7].corr())['Close_y']['Close_x'], 3)
        time_df = temp_merged_df.iloc[x:x+5][['Date', 'Week']]
        time_df['Corr'] = corr_value
        corr_df = corr_df.append(time_df)

    corr_df['Gen_Corr'] = gen_corr
    
    # Setting 2nd df for rest of analysis
    merged_df = (merged_df.drop(columns=['Open_x', 'High_x', 'Low_x', 'Volume_x', 'Dividends_x'
                         , 'Open_y', 'High_y', 'Low_y', 'Volume_y', 'Dividends_y',
                         'Stock Splits_y', 'Stock Splits_x']))
    merged_df['Close_y'] = merged_df['Close_y'] / 10
    merged_df['ratio'] = merged_df['Close_x'][0] / merged_df['Close_y'][0]
    merged_df['mod_Close_y'] = merged_df['Close_y'] * (merged_df['ratio'].mean()*1.1)
    
    corr_df['Gen_Corr'] = gen_corr
    
    display(corr_with_sp500(corr_df, merged_df, gen_corr, symbol_a, symbol_b))

    # Continuationg to reaction of stock to S&P major actions
    
    merged_df = merged_df.tail(365)
    merged_df['Year'] = merged_df['Date'].apply(lambda x : x.year)
    merged_df['Month'] = merged_df['Date'].apply(lambda x : x.month)
    merged_df['Week'] = merged_df['Date'].apply(lambda x : x.week)
    merged_df = merged_df.groupby(['Year', 'Month', 'Week'])['Close_x', 'Close_y'].mean().reset_index()
    
    merged_df = merged_df.rename(columns={'Close_x' : f'Close_{symbol_a}', 'Close_y' : f'Close_{symbol_b}'})
    
    comp_sp500 = merged_df.tail(length-1).reset_index(drop=True)
    comp_sp500['Week'] = comp_sp500['Week'].apply(lambda x : x + 1)

    merged_df = (pd.merge(merged_df,
          comp_sp500,
          on =['Year', 'Month', 'Week'],
          how ='inner'))
    merged_df[f'Weekly_dif_{symbol_b}'] = merged_df[f'Close_{symbol_b}_x'] / merged_df[f'Close_{symbol_b}_y']
    merged_df[f'Weekly_dif_{symbol_a}'] = merged_df[f'Close_{symbol_a}_x'] / merged_df[f'Close_{symbol_a}_y']
    
    largest_lst = merged_df[f'Weekly_dif_{symbol_b}'].nlargest(5).reset_index(drop=True).values.tolist()
    samllest_lst = merged_df[f'Weekly_dif_{symbol_b}'].nsmallest(5).reset_index(drop=True).values.tolist()
    joinedlist = samllest_lst + largest_lst
    
    # Table show how did the stock reacted when S&P500 did a major moves
    merged_df = (merged_df.loc[merged_df[f'Weekly_dif_{symbol_b}'].isin(joinedlist)]
         .reset_index(drop=True)
         .drop(columns=[f'Close_{symbol_a}_x', f'Close_{symbol_b}_x', f'Close_{symbol_a}_y', f'Close_{symbol_b}_y']))
    merged_df[f'Weekly_dif_{symbol_b}'] = 1 - merged_df[f'Weekly_dif_{symbol_b}']
    merged_df[f'Weekly_dif_{symbol_a}'] = 1 - merged_df[f'Weekly_dif_{symbol_a}']
    merged_df[f'abs_dif_{symbol_a}_{symbol_b}'] = round(abs(merged_df[f'Weekly_dif_{symbol_a}'] - merged_df[f'Weekly_dif_{symbol_b}']), 3)
    merged_df['text_loc'] = (merged_df[f'Weekly_dif_{symbol_a}'] + merged_df[f'Weekly_dif_{symbol_b}']) / 2
#     merged_df['text_loc'] = max((merged_df[f'Weekly_dif_{symbol}'] + merged_df['Weekly_dif_S&P500']) / 2)

    display(major_moves_corr(merged_df, symbol_a, symbol_b))
    

In [103]:
# corr_df['Gen_Corr'] = gen_corr
def corr_with_sp500(corr_df, merged_df, gen_corr, symbol_a, symbol_b):
    chart1 = alt.Chart(corr_df, title = f'Correlation for {symbol_b} & {symbol_a}').mark_line().encode(x='Date:T', y='Corr:Q').properties(width=400)
    chart2 = alt.Chart(corr_df).mark_line(color='#737373').encode(x='Date:T', y='Gen_Corr:Q').properties(width=400)

    corr_with_sp500 = chart1 + chart2
    
    chart3 = alt.Chart(merged_df, title = f'Correlation for {symbol_b} (Blue) & {symbol_a} (Red)').mark_line().encode(
    x='Date:T', y=alt.Y('mod_Close_y:Q', title=f'Price')).properties(width=400)
    chart4 = alt.Chart(merged_df).mark_line(color='#990000').encode(x='Date:T', y='Close_x:Q').properties(width=400)
    
    corr_with_sp500_price = chart3 + chart4
    
    if gen_corr > 0:
        message = f"The weekly correlation incline is positive ({round(gen_corr, 3)})."
        print(message)
    else:
        message = f"The weekly correlation incline is negative ({round(gen_corr, 3)})."
        print(message)
    message = f"The median weekly correlation is ({corr_df['Corr'].median()})."
    print(message)
    
    return(corr_with_sp500 | corr_with_sp500_price)

In [104]:
def major_moves_corr(merged_df, symbol_a, symbol_b):
    chart5 = (alt.Chart(merged_df, title = f"10 Biggest Prices Changes for {symbol_b}, {symbol_a}'s Correlation to them and % Change Gap")
              .mark_line()
              .encode(
              x='Week', y=alt.Y(f'Weekly_dif_{symbol_b}:Q', title=f'Weely Price Change',
                               axis=alt.Axis(format='%'))).properties(width=850))
    chart6 = (alt.Chart(merged_df).mark_point(size=30, color='#990000').encode(x='Week', y=f'Weekly_dif_{symbol_a}')
             .properties(width=850))
    
    annotation = alt.Chart(merged_df).mark_text(
        align='center',
        baseline='top',
        fontSize = 12,
        dx = 0
        ).encode(
            x='Week',
            y='text_loc:Q',
            text=alt.Text(f'abs_dif_{symbol_a}_{symbol_b}:Q', format='.0%')
        ).properties(width=850)

    corr_with_sp500_price = chart5 + chart6 + annotation
    return(corr_with_sp500_price)

# major_moves_corr(merged_df, symbol_a)

In [122]:
def main_plots(symbol_a, symbol_b, amt_invested, start_date_yyyy_dd_mm):
    df, df_dup, div_yield_df, div_growth_df = basic_analysis_func(symbol_a, start_date_yyyy_dd_mm)
#     further_analysis(symbol_a)
#     calc_yield_func(amt_invested, start_date_yyyy_dd_mm, symbol_a)
#     next_pay_date(symbol_a, start_date_yyyy_dd_mm)
    correlation_analysis_365(symbol_a, symbol_b)
    
main_plots('AAPL', '^GSPC', 10000, '2015-01-01')

# TODO - Error while trying to analyze on specific stocks 'NoneType' object is not iterable

alt.HConcatChart(...)

alt.HConcatChart(...)

The weekly correlation incline is positive (0.926).
The median weekly correlation is (0.816).


alt.HConcatChart(...)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:58: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


alt.LayerChart(...)

In [125]:
def main_funds(symbol_a, symbol_b, amt_invested, start_date_yyyy_dd_mm):
    df, df_dup, div_yield_df, div_growth_df = basic_analysis_func_no_disp(symbol_a, start_date_yyyy_dd_mm)
    further_analysis(symbol_a)
    calc_yield_func(amt_invested, start_date_yyyy_dd_mm, symbol_a)
    next_pay_date(symbol_a, start_date_yyyy_dd_mm)
#     correlation_analysis_365(symbol_a, symbol_b)
    
main_funds('AAPL', '^GSPC', 10000, '2018-01-01')

# TODO - fix issue with מדדים

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


alt.HConcatChart(...)

Next dividend approximated pay-day is : 8-2-2021


In [87]:
pd.read_html('https://finance.yahoo.com/quote/XOM/key-statistics?p=XOM') # goes 1 - 9

[                             0        1
 0      Market Cap (intraday) 5  197.08B
 1           Enterprise Value 3  257.11B
 2                 Trailing P/E    59.91
 3                Forward P/E 1    23.78
 4  PEG Ratio (5 yr expected) 1   -10.76
 5            Price/Sales (ttm)     1.01
 6             Price/Book (mrq)     1.11
 7   Enterprise Value/Revenue 3     1.31
 8    Enterprise Value/EBITDA 6    13.77,
                           0        1
 0         Beta (5Y Monthly)     1.34
 1          52-Week Change 3  -25.10%
 2   S&P500 52-Week Change 3   14.44%
 3            52 Week High 3    63.01
 4             52 Week Low 3    30.11
 5   50-Day Moving Average 3    44.75
 6  200-Day Moving Average 3    39.90,
                                                  0       1
 0                              Avg Vol (3 month) 3  31.08M
 1                               Avg Vol (10 day) 3  28.51M
 2                             Shares Outstanding 5   4.23B
 3                                          

Next step - find the fundamental ways to check it.

### Warren's buffet method of analysis

In [107]:
pd.read_html('https://finance.yahoo.com/quote/XOM/key-statistics?p=XOM')[0] # goes 1 - 9

,0,1
0,Market Cap (intraday) 5,203.65B
1,Enterprise Value 3,267.91B
2,Trailing P/E,NaN
3,Forward P/E 1,15.08
4,PEG Ratio (5 yr expected) 1,1.72
5,Price/Sales (ttm),1.12
6,Price/Book (mrq),1.15
7,Enterprise Value/Revenue 3,1.48
8,Enterprise Value/EBITDA 6,-29.43


In [120]:
# Getting the P/E
# Note - P/E == trailing P/E
fundamentals_df = pd.read_html(f'https://finance.yahoo.com/quote/{symbol}/key-statistics?p={symbol}')[0]

# PE
trailing_pe = fundamentals_df[1][2]

# Assessed future earrings
forward_pe = fundamentals_df[1][3]

# Price/book
price_book = fundamentals_df[1][6]

# def calc_func(trailing_pe, forward_pe, price_book):
#     if trailing_pe.isnan:
        
type(forward_pe)

    



str